In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
generative = smc.ParticleTrace(1)

In [4]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [5]:
ssm_step = combinators.Model(ssm.ssm_step)
ssm_step.condition(trace=generative, observations=None)

In [6]:
for t in range(zs.shape[1] - 1):
    zs[:, t+1], mu, sigma, delta = ssm_step(zs[:, t], mu, sigma, delta, t)

In [7]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [8]:
num_particles = 2500

In [9]:
smc_runner = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [10]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [11]:
inference, init_ssm = smc.variational_smc(num_particles, ssm.init_ssm, smc_runner, 500, 50, ssm_params, generative)

07/06/2018 17:53:10 Variational SMC ELBO=-3.9120e+02 at epoch 0
07/06/2018 17:53:11 Variational SMC ELBO=-3.9120e+02 at epoch 1
07/06/2018 17:53:12 Variational SMC ELBO=-3.9120e+02 at epoch 2
07/06/2018 17:53:13 Variational SMC ELBO=-3.9120e+02 at epoch 3
07/06/2018 17:53:14 Variational SMC ELBO=-3.9120e+02 at epoch 4
07/06/2018 17:53:15 Variational SMC ELBO=-3.9120e+02 at epoch 5
07/06/2018 17:53:16 Variational SMC ELBO=-3.9120e+02 at epoch 6
07/06/2018 17:53:18 Variational SMC ELBO=-3.9120e+02 at epoch 7
07/06/2018 17:53:19 Variational SMC ELBO=-3.9120e+02 at epoch 8
07/06/2018 17:53:20 Variational SMC ELBO=-3.9120e+02 at epoch 9
07/06/2018 17:53:21 Variational SMC ELBO=-3.9120e+02 at epoch 10
07/06/2018 17:53:22 Variational SMC ELBO=-3.9120e+02 at epoch 11
07/06/2018 17:53:24 Variational SMC ELBO=-3.9120e+02 at epoch 12
07/06/2018 17:53:25 Variational SMC ELBO=-3.9120e+02 at epoch 13
07/06/2018 17:53:26 Variational SMC ELBO=-3.9120e+02 at epoch 14
07/06/2018 17:53:27 Variational SMC

In [12]:
inference

Trace{'mu': Normal([torch.cuda.FloatTensor of size 2500]), 'sigma': Normal([torch.cuda.FloatTensor of size 2500]), 'delta': Normal([torch.cuda.FloatTensor of size 2500]), 'Z_0': Normal([torch.cuda.FloatTensor of size 2500]), 'Z_1': Normal([torch.cuda.FloatTensor of size 2500]), 'X_1': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_2': Normal([torch.cuda.FloatTensor of size 2500]), 'X_2': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_3': Normal([torch.cuda.FloatTensor of size 2500]), 'X_3': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_4': Normal([torch.cuda.FloatTensor of size 2500]), 'X_4': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_5': Normal([torch.cuda.FloatTensor of size 2500]), 'X_5': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_6': Normal([torch.cuda.FloatTensor of size 2500]), 'X_6': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_7': Normal([torch.cuda.FloatTensor of size 2500]), 'X_7': Normal([torch.cuda.FloatTensor of size 250

In [13]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [14]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 4.058616
SMC MSE at time 2: 1.140794
SMC MSE at time 3: 2.148808
SMC MSE at time 4: 1.085691
SMC MSE at time 5: 1.182723
SMC MSE at time 6: 1.816136
SMC MSE at time 7: 1.578521
SMC MSE at time 8: 0.873339
SMC MSE at time 9: 2.260153
SMC MSE at time 10: 0.838170
SMC MSE at time 11: 0.751883
SMC MSE at time 12: 1.214239
SMC MSE at time 13: 1.023255
SMC MSE at time 14: 0.372340
SMC MSE at time 15: 1.574463
SMC MSE at time 16: 1.219577
SMC MSE at time 17: 0.523216
SMC MSE at time 18: 3.291710
SMC MSE at time 19: 10.757235
SMC MSE at time 20: 5.925147
SMC MSE at time 21: 1.663482
SMC MSE at time 22: 0.994209
SMC MSE at time 23: 1.796256
SMC MSE at time 24: 1.118500
SMC MSE at time 25: 1.859257
SMC MSE at time 26: 0.995637
SMC MSE at time 27: 0.972305
SMC MSE at time 28: 1.190484
SMC MSE at time 29: 0.496975
SMC MSE at time 30: 0.787951
SMC MSE at time 31: 0.890814
SMC MSE at time 32: 1.514876
SMC MSE at time 33: 2.183378
SMC MSE at time 34: 2.503562
SMC MSE at time 35: 5.